In [1]:
import numpy as np
import pandas as pd
import re

import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

C:\Users\smvan\anaconda3\lib\site-packages\pdpipe\col_generation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
consolidated_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/Consolidated_purchases/OK_consol_purchases_march1_april21.csv")
consolidated_df.head()

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,prod_group_descrip,order_code,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount
0,252033,M123033004,-172.0,69025,-59.0,-123.9,-123.9,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
1,252099,0252099,153.3,69025,15.0,31.5,31.5,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
2,252100,0252099,-153.3,69025,-15.0,-31.5,-31.5,each,2.1,CANDIED CINNAMON ALMOND,...,CANDIES SNACKS,KK051094,N,1.0,1.0,each,7,Candy,N,0
3,252099,0252099,153.3,68933,14.0,29.4,29.4,each,2.1,CANDIED COCONUT ALMO,...,CANDIES SNACKS,KK051093,N,1.0,1.0,each,7,Candy,N,0
4,252100,0252099,-153.3,68933,-14.0,-29.4,-29.4,each,2.1,CANDIED COCONUT ALMO,...,CANDIES SNACKS,KK051093,N,1.0,1.0,each,7,Candy,N,0


In [3]:
categories_df = pd.read_csv("C:/Users/smvan/CFFS-S23/CFFS-22-23/data/GHG_categories/ghge_factors.csv")
categories_df


,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316
...,...,...,...
57,58,sauces & paste,0.0000
58,59,manually adjusted,0.0000
59,60,human labor,0.0000
60,61,kitchen supplies,0.0000


In [4]:
# Convert the "item_descrip" and "Food Category" columns to lowercase
consolidated_df['item_descrip'] = consolidated_df['item_descrip'].str.lower()
categories_df['Food Category'] = categories_df['Food Category'].str.lower()

# Initialize new columns
consolidated_df['Food Category'] = np.nan
consolidated_df['Category ID'] = np.nan

# Define a function to split the Food Category column
def split_food_category(category):
    separators = ['/', '(', ',', ')']
    for separator in separators:
        category = category.replace(separator, ' ')
    return category.strip()

for index, row in categories_df.iterrows():
    categories_df.at[index, 'Food Category'] = split_food_category(row['Food Category'])
    
# Iterate through each row
for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    
    # Split the item_descrip into words
    item_words = item_descrip.split()
    
    # Find matches between the words in item_descrip and the Food Category column
    matches = categories_df[categories_df['Food Category'].apply(lambda x: any(word in x.split() for word in item_words))]
    
    # If there is a match, assign the corresponding values
    if not matches.empty:
        consolidated_df.at[index, 'Food Category'] = matches['Food Category'].iloc[0]
        consolidated_df.at[index, 'Category ID'] = matches['Category ID'].iloc[0]

# Print the updated DataFrame
consolidated_df





,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
0,252033,M123033004,-172.00,69025,-59.0,-123.90,-123.90,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,almond milk,27.0
1,252099,0252099,153.30,69025,15.0,31.50,31.50,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,almond milk,27.0
2,252100,0252099,-153.30,69025,-15.0,-31.50,-31.50,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,almond milk,27.0
3,252099,0252099,153.30,68933,14.0,29.40,29.40,each,2.10,candied coconut almo,...,N,1.0,1.0,each,7,Candy,N,0,NaN,NaN
4,252100,0252099,-153.30,68933,-14.0,-29.40,-29.40,each,2.10,candied coconut almo,...,N,1.0,1.0,each,7,Candy,N,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18381,252537,9001537618,3899.87,5528,4.0,208.92,208.92,cs,52.23,wrap film roll 17x2500 ct box,...,N,2500.0,1.0,cs,4,Misc.,N,0,NaN,NaN
18382,252458,9001537197,7606.70,5528,1.0,52.23,52.23,cs,52.23,wrap film roll 17x2500 ct box,...,N,2500.0,1.0,cs,4,Misc.,N,0,NaN,NaN
18383,250022,9001206712,8283.19,30507,3.0,225.57,225.57,cs,75.19,"wrap foil hd 18"" 45cmx100m",...,N,328.0,1.0,cs,4,Misc.,N,0,NaN,NaN
18384,250792,9001333265,2352.89,30507,1.0,75.19,75.19,cs,75.19,"wrap foil hd 18"" 45cmx100m",...,N,328.0,1.0,cs,4,Misc.,N,0,NaN,NaN


In [5]:
# for index, row in categories_df.iterrows():
#     for i, r in consolidated_df.iterrows():
#         if consolidated_df.loc[i, 'Food Category'].str.contains(categories_df.loc[index, 'Food Category']):
#             consolidated_df.loc[index, 'Food Category'] = categories_df.loc[index, 'Food Category']
#             consolidated_df.loc[index, 'Category ID'] = categories_df.loc[index, 'Category ID']

In [6]:
# if consolidated_df.loc[814, 'Food Category'].str.contains(categories_df.loc[index, 'Food Category']):
#     print("yes")
# else:
#     print("no")
    
# consolidated_df.loc[814, 'Food Category']

In [7]:
# Iterate through each row
for index, row in consolidated_df.iterrows():
    item_descrip = row['item_descrip']
    
    # Split the item_descrip into words
    item_words = item_descrip.split()
    
    # Check if prod_group_descrip is "SUPPLY"
    if row['prod_group_descrip'] == 'SUPPLY':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61
        
    if row['prod_group_descrip'] == 'DISPOSABLES':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61

    if row['prod_group_descrip'] == 'CLEANING':
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61
        
    if row['prod_group_descrip'] == 'BREAD':
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24   
    
    if row['prod_group_descrip'] == 'BAKED GOODS':
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
        
    if row['prod_group_descrip'] == 'CANDIES SNACKS':
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'Category ID'] = 43
    
    
    elif 'oat' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'oats (oatmeal)'
        consolidated_df.at[index, 'Category ID'] = 23

    elif 'dress' in item_words or 'sauce' in item_words or 'jelly' in item_words or 'ketchup' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sauces & paste'
        consolidated_df.at[index, 'Category ID'] = 58

    elif 'mayonaise*vegan' in item_words or 'mayonnaise' in item_words or 'mustard' in item_words or 'spread' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sauces & paste'
        consolidated_df.at[index, 'Category ID'] = 58
        
        

    # Check if item_descrip contains "cereal"
    elif 'cereal' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other grains/cereals'
        consolidated_df.at[index, 'Category ID'] = 21
    
    # Check if item_descrip contains "proscuitto" or "ham"
    elif 'prosciutto' in item_words or 'ham' in item_words or 'bacon' in item_words or 'chorizo' in item_words or 'sausage' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'pork (pig meat)'
        consolidated_df.at[index, 'Category ID'] = 3
    
    # Check if item_descrip contains "salami"
    elif 'salami' in item_words or '50/50' in item_words or 'patties' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'beef & buffalo meat'
        consolidated_df.at[index, 'Category ID'] = 1
        
    elif 'dough' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24

    elif 'juice' in item_words or 'pop' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'Category ID'] = 55       

    elif 'coconut' in item_words or 'apricot' in item_words or 'mango-tamarind' in item_words or 'cucumber' in item_words or 'peppers' in item_words or 'salsa' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 

    elif 'grapes' in item_words or 'kiwi' in item_words or 'mangoes' in item_words or 'honeydew' in item_words or 'peach' in item_words or 'pears' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 
    
    elif 'squash' in item_words or 'watermelon' in item_words or 'zucchini' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 
        
    elif 'muffin' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24

    elif 'paneer' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cheese'
        consolidated_df.at[index, 'Category ID'] = 6       

    elif ('cream' in item_words or 'creamo10%' in item_words) and not 'ice' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cream'
        consolidated_df.at[index, 'Category ID'] = 8  
        
        
    elif ('cream' in item_words and 'ice' in item_words) or 'magnum' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'ice cream'
        consolidated_df.at[index, 'Category ID'] = 7  
        
        
    elif 'pie' in item_words or 'waffle' in item_words or 'pastry' in item_words or 'flour' in item_words or 'bun' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
    
    elif 'bar' in item_words or 'biscuits' in item_words or 'brownie' in item_words or 'cake' in item_words or 'cookie' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
        
    elif 'crepe' in item_words or 'croissant' in item_words or 'danish' in item_words or 'donut' in item_words or 'eclair' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
    
    elif 'loaf' in item_words or 'pretzel' in item_words or 'scone' in item_words or 'tart' in item_words or 'tiramisu' in item_words or 'lady' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24

    elif 'crouton' in item_words or 'ck|prep|granola' in item_words or 'noodles' in item_words or 'perogy' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wheat/rye (bread, pasta, baked goods)'
        consolidated_df.at[index, 'Category ID'] = 24
        
        
    
    elif row['prod_group_descrip'] == 'BEVERAGE' and pd.isna(row['Food Category']):
        consolidated_df.at[index, 'Food Category'] = 'water & beverages'
        consolidated_df.at[index, 'Category ID'] = 55
    
               
    elif 'co2' in item_words or 'soda' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'kitchen supplies'
        consolidated_df.at[index, 'Category ID'] = 61
        
    elif 'sugar' in item_words or 'melona' in item_words or 'honey' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'sugars and sweeteners'
        consolidated_df.at[index, 'Category ID'] = 43 
    
    elif ('chip' in item_words and pd.isna(row['Food Category'])) or 'potato' in item_words or 'fries' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'potatoes'
        consolidated_df.at[index, 'Category ID'] = 41
        
    elif 'hashbrown' in item_words or 'potato*pompom' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'potatoes'
        consolidated_df.at[index, 'Category ID'] = 41

    elif 'salt&vin' in item_words or 'balsamic' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'vinegar'
        consolidated_df.at[index, 'Category ID'] = 57
    
    elif 'egg' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'eggs'
        consolidated_df.at[index, 'Category ID'] = 11
    
    elif 'eggplant' in item_words or 'kale' in item_words or 'lettuce' in item_words or 'romaine' in item_words or 'salad':
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
 
    elif 'chocolate' in item_words or 'arnotts' in item_words or 'choco' in item_words or 'cho' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'cocoa'
        consolidated_df.at[index, 'Category ID'] = 52
    
    elif 'dragon' in item_words or 'california' in item_words or 'shrimpmt' in item_words or 'prawn' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'crustaceans (shrimp/prawns)'
        consolidated_df.at[index, 'Category ID'] = 13


    elif 'milkettes' in item_words:
        consolidated_df.at[index, 'Food Category'] = "milk (cow's milk)"
        consolidated_df.at[index, 'Category ID'] = 9
    
    elif 'rubbed' in item_words or 'pepper' in item_words or 'powder' in item_words or 'masala' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54

    elif 'cumin' in item_words or 'cloves' in item_words or 'ground' in item_words or 'seasoning' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54
        
    elif 'blend' in item_words or 'chili' in item_words or 'pwdr' in item_words or 'caper' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54
    
    elif 'leaf' in item_words or 'chile' in item_words or 'paprika' in item_words or 'leaves' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54   
        
    elif 'anise' in item_words or 'cardamom' in item_words or 'mix' in item_words:
        consolidated_df.at[index, 'Food Category'] = "stimulants & spices misc."
        consolidated_df.at[index, 'Category ID'] = 54 

    elif 'base' in item_words and 'chick' in item_words or 'chick' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'poultry (chicken, turkey)'
        consolidated_df.at[index, 'Category ID'] = 4
        

    elif 'base' in item_words and 'veg' in item_words or 'veg' in item_words or 'seaweed' in item_words or 'ck|g&g|salad|kalecaesarw/chick' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
        
    elif 'nori' in item_words or 'ck|g&g|salad|kale caesar.' in item_words or 'pickle' in item_words or 'choy' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40

        
    elif 'spinach' in item_words or 'carrot' in item_words or 'artichoke' in item_words or 'avocado' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
        
    elif 'onion' in item_words or 'leeks' in item_words or 'onions' in item_words or 'shallots' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'onions and leeks'
        consolidated_df.at[index, 'Category ID'] = 39
        
    elif 'broccoli' in item_words or 'brussel' in item_words or 'cabbage' in item_words or 'cantaloupe' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
    
    elif 'carrots' in item_words or 'cauliflower' in item_words or 'celery' in item_words or 'coleslaw' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
    
    elif 'sprouts' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other vegetables'
        consolidated_df.at[index, 'Category ID'] = 40
        
    
    elif 'pumpkin' in item_words or 'jackfruit' in item_words or 'pineapple' in item_words or 'raisin' in item_words or 'cherries' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other fruits'
        consolidated_df.at[index, 'Category ID'] = 31 
        
    elif 'pumpkin' in item_words and 'kern' in item_words or 'quinoa' in item_words or 'sesame' in item_words or 'seed' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tree nuts and seeds'
        consolidated_df.at[index, 'Category ID'] = 26 
        
    elif 'pumpkin' in item_words or 'hemp' in item_words or 'nut' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tree nuts and seeds'
        consolidated_df.at[index, 'Category ID'] = 26

    elif 'tomato' in item_words or 'tomatillos' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'tomatoes'
        consolidated_df.at[index, 'Category ID'] = 37
        
    elif 'soybean' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'soybeans/tofu'
        consolidated_df.at[index, 'Category ID'] = 20
    
    elif 'sake' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'wine grapes (wine)'
        consolidated_df.at[index, 'Category ID'] = 51
    
    elif 'bean' in item_words or 'vanilla' in item_words or 'lentils' in item_words or 'ck|g&g|salad|medchickpea' in item_words or 'tempeh' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'other legumes'
        consolidated_df.at[index, 'Category ID'] = 16
    
    elif 'chickpea.' in item_words or 'falafel' in item_words or ('patties' in item_words and 'vegan' in item_words):
        consolidated_df.at[index, 'Food Category'] = 'other legumes'
        consolidated_df.at[index, 'Category ID'] = 16
    
    
    elif 'tuna' in item_words or 'squid' in item_words or 'cod' in item_words or 'halibut' in item_words or 'salmon' in item_words or 'clam' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'fish (finfish)'
        consolidated_df.at[index, 'Category ID'] = 12
    
    elif 'raspberry' in item_words or 'blackberry' in item_words or 'blueberry' in item_words or 'strawberry' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'berries'
        consolidated_df.at[index, 'Category ID'] = 34
        
    elif 'raspberries' in item_words or 'blackberries' in item_words or 'blueberries' in item_words or 'strawberries' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'berries'
        consolidated_df.at[index, 'Category ID'] = 34
    
    elif 'sal' in item_words or 'samosa' in item_words or 'wrap' in item_words or 'sandwich' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'manually adjusted'
        consolidated_df.at[index, 'Category ID'] = 59
 
    elif 'mandarin' in item_words or 'grapefruit' in item_words or 'lemons' in item_words or 'limes' in item_words or 'mandarins' in item_words or 'oranges' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'citrus fruit'
        consolidated_df.at[index, 'Category ID'] = 35
    
    elif 'olive' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'olives (oil)'
        consolidated_df.at[index, 'Category ID'] = 49
    
    elif 'beets' in item_words or 'daikon' in item_words or 'dill' in item_words or 'ginger' in item_words or 'garlic' in item_words or 'jicama' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'root vegetables'
        consolidated_df.at[index, 'Category ID'] = 38
   
    elif 'parsnip' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'root vegetables'
        consolidated_df.at[index, 'Category ID'] = 38
        
        
    elif 'banana' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'bananas'
        consolidated_df.at[index, 'Category ID'] = 33
        
    elif 'thyme' in item_words or 'herb' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'herbs'
        consolidated_df.at[index, 'Category ID'] = 63    
        
    elif 'pea' in item_words:
        consolidated_df.at[index, 'Food Category'] = 'peas'
        consolidated_df.at[index, 'Category ID'] = 18 
        
# consolidated_df
# 18386

#     elif 'chip' in item_words or 'potato' in item_words:
#         consolidated_df.at[index, 'Food Category'] = 'potatoes'
#         consolidated_df.at[index, 'Category ID'] = 41
# Print the updated DataFrame

In [11]:
filtered_df = consolidated_df[~consolidated_df["Category ID"].isna()]
filtered_df


,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID
0,252033,M123033004,-172.00,69025,-59.0,-123.90,-123.90,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
1,252099,0252099,153.30,69025,15.0,31.50,31.50,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
2,252100,0252099,-153.30,69025,-15.0,-31.50,-31.50,each,2.10,candied cinnamon almond,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
3,252099,0252099,153.30,68933,14.0,29.40,29.40,each,2.10,candied coconut almo,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
4,252100,0252099,-153.30,68933,-14.0,-29.40,-29.40,each,2.10,candied coconut almo,...,N,1.0,1.0,each,7,Candy,N,0,sugars and sweeteners,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18381,252537,9001537618,3899.87,5528,4.0,208.92,208.92,cs,52.23,wrap film roll 17x2500 ct box,...,N,2500.0,1.0,cs,4,Misc.,N,0,other vegetables,40.0
18382,252458,9001537197,7606.70,5528,1.0,52.23,52.23,cs,52.23,wrap film roll 17x2500 ct box,...,N,2500.0,1.0,cs,4,Misc.,N,0,other vegetables,40.0
18383,250022,9001206712,8283.19,30507,3.0,225.57,225.57,cs,75.19,"wrap foil hd 18"" 45cmx100m",...,N,328.0,1.0,cs,4,Misc.,N,0,other vegetables,40.0
18384,250792,9001333265,2352.89,30507,1.0,75.19,75.19,cs,75.19,"wrap foil hd 18"" 45cmx100m",...,N,328.0,1.0,cs,4,Misc.,N,0,other vegetables,40.0


In [9]:
unfiltered_df = consolidated_df[consolidated_df["Category ID"].isna()]
unfiltered_df

,invoice_counter,invoice_num,invoice_total,item_num,qty,line_amount,adj_line_amount,uom,unit_cost,item_descrip,...,compute_0022,pak_qty,case_qty,purchase_uom,cat_num,cat_descrip,has_credit,credit_pak_amount,Food Category,Category ID


In [10]:
# # Standard unit converter:
# # Construct a unit converter for specific ingredients

# # After this line below, spc_cov contains only the non-empty values from the 'ConversionId' column of the Conversions DataFrame.
# spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))


# # Comments for spc_converter:
# # The function checks if ingredient is in the liquid_unit or solid_unit lists. If so, it calls std_converter(qty, uom) to 
# # convert the quantity and UOM to a standardized unit.

# # If uom is not in liquid_unit or solid_unit it checks if ingre is in spc_cov, if it is and the ConvertToUom is equal to grams 
# # then the function applies the factor to the qty argument to convert it to the standardized unit, and returns the result as
# # a tuple containing the converted quantity and uom. If no conversion found, then it calls std_converter(qty, uom)

# # If uom not in liquid_unit or solid_unit and if ingre is not in spc_cov then the function calls std_converter(qty, uom)

# def spc_converter(ingre, qty, uom):
#     if uom in liquid_unit + solid_unit:
#         return std_converter(qty, uom)
#     elif ingre in spc_cov:
#         conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
#                                     & (Conversions['ConvertToUom'] == 'g')]
#         multiplier = conversion['Multiplier']
#         if multiplier.empty:
#             return std_converter(qty, uom)
#         else: 
#             Qty = float(qty)/float(multiplier)
#             Uom = conversion['ConvertToUom'].values[0]
#             return (Qty, Uom)
#     else:
#         return std_converter(qty, uom)